## Preprocessing

In [40]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
import keras_utils
from tensorflow_serving.session_bundle import exporter
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils




ModuleNotFoundError: No module named 'tensorflow_serving.session_bundle'

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [10]:

#  Import and read the charity_data.csv.
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [13]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# Choose a cutoff value and create a list of application types to be replaced
application_df['APPLICATION_TYPE'].value_counts().median()
cutoff = 725
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df[application_df['APPLICATION_TYPE'].map(application_df['APPLICATION_TYPE'].value_counts()) < cutoff]
application_types_to_replace = application_types_to_replace['APPLICATION_TYPE'].unique().tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [16]:
# You may find it helpful to look at CLASSIFICATION value counts >1
temp_df = application_df[application_df['CLASSIFICATION'].map(application_df['CLASSIFICATION'].value_counts()) > 1]
temp_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [17]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 194
classifications_to_replace = application_df[application_df['CLASSIFICATION'].map(application_df['CLASSIFICATION'].value_counts()) < cutoff]
classifications_to_replace = classifications_to_replace['CLASSIFICATION'].unique().tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
# Split our preprocessed data into our features and target arrays
temp_X = dummies.drop('IS_SUCCESSFUL', axis=1)
temp_Y = dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(temp_X, temp_Y, random_state=1)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=45))

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
nn.add(tf.keras.layers.Dense(units=3))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 230       
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


2022-11-15 19:49:13.156806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, Y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 4.9155 - accuracy: 0.4889
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 2.5513 - accuracy: 0.4986
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 2.0969 - accuracy: 0.4662
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 1.8787 - accuracy: 0.4577
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 1.6078 - accuracy: 0.4762
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 1.4787 - accuracy: 0.4492
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 1.3720 - accuracy: 0.4204
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 1.1818 - accuracy: 0.4372
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 1.1025 - accuracy: 0.4346
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 1.0520 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5960 - accuracy: 0.5283
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5956 - accuracy: 0.3160
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5967 - accuracy: 0.4033
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5952 - accuracy: 0.4037
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5997 - accuracy: 0.4585
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5971 - accuracy: 0.3489
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5962 - accuracy: 0.3441
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5956 - accuracy: 0.3864
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5959 - accuracy: 0.4079
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5950 - accuracy: 

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6301 - accuracy: 0.3867 - 236ms/epoch - 881us/step
Loss: 0.6300737857818604, Accuracy: 0.38670554757118225


In [32]:
K.set_learning_phase(0)  # all new operations will be in test mode from now on Option A

# serialize the model and get its weights, for quick re-building
config = previous_model.get_config()
weights = previous_model.get_weights()

# re-build a model where the learning phase is now hard-coded to 0
from keras.models import model_from_config
new_model = model_from_config(config)
new_model.set_weights(weights)






NameError: name 'K' is not defined

In [37]:
export_path = ("./Resources/weights.hdf5") # where to save the exported graph  Option B
export_version = (1)
saver = tf.keras.Model.save_weights(share=True)
model_exporter = exporter.Exporter(saver)
signature = exporter.classification_signature(input_tensor=model.input,
                                              scores_tensor=model.output)
model_exporter.init(sess.graph.as_graph_def(),
                    default_graph_signature=signature)
model_exporter.export(export_path, tf.constant(export_version), sess)




TypeError: save_weights() got an unexpected keyword argument 'shared'

In [34]:
# Export our model to HDF5 file   Option C

output_tensor = model.output

with open(model_save_file, "w") as model_save_file:
    model_json = model.to_json()
    model_save_file.write(model_json)

# serialize weights to HDF5    
model.save_weights(model_weights_save_file)
    


NameError: name 'model' is not defined

In [39]:
# Create a saver.  
saver = tf.compat.v1.train.Saver(share=True)
# Launch the graph and train, saving the model every 1,000 steps.
sess = tf.compat.v1.Session()
for step in range(1000000):
    sess.run(training_op)
    if step % 1000 == 0:
        # Append the step number to the checkpoint name:
        saver.save(sess, 'my-model', global_step=step)

TypeError: __init__() got an unexpected keyword argument 'shared'